In [8]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os

from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from bs4 import BeautifulSoup


 

stemmer=SnowballStemmer("english")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

import networkx as nx
from collections import defaultdict
from itertools import combinations

from fa2_modified import ForceAtlas2
from community import community_louvain
from operator import itemgetter

from graspologic.partition import leiden, modularity
from graspologic.utils import is_symmetric

import textmining
  

%env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [10]:
path = %env folder

In [12]:
# Load the GEXF file
G = nx.read_gexf(path+R'processed_data/co_word_network_+2tfidf_backbone_90.gexf')


In [13]:
components = nx.connected_components(G)
num_components = nx.number_connected_components(G)

# Get the largest connected component
largest_component = max(components, key=len)

G_mc = G.subgraph(largest_component)

In [14]:
num_components

2

In [18]:
resolutions = []
community_nrs = []
modularity_scores = []
seed_nrs = []

for loop_nr in list(range(1,101,1)):  
    for resolution in [x/10 for x in list(range(5, 16,1))]:
        leiden_partition_map = leiden(G_mc, resolution=resolution, random_seed = loop_nr)
        community_nr = len(set(leiden_partition_map.values()))
        modularity_score = modularity(G_mc, leiden_partition_map)
        print(f"Resolution: {resolution}, Modularity: {modularity_score}, Nr of communities: {community_nr}, Seed nr: {loop_nr}")
        resolutions.append(resolution)
        community_nrs.append(community_nr)
        modularity_scores.append(modularity_score)
        seed_nrs.append(loop_nr)
        
    

Resolution: 0.5, Modularity: 6.574162718404933e-06, Nr of communities: 2, Seed nr: 1
Resolution: 0.6, Modularity: 0.23393596855163584, Nr of communities: 7, Seed nr: 1
Resolution: 0.7, Modularity: 0.24723348800455322, Nr of communities: 10, Seed nr: 1
Resolution: 0.8, Modularity: 0.3068370236698124, Nr of communities: 8, Seed nr: 1
Resolution: 0.9, Modularity: 0.31675441540485233, Nr of communities: 10, Seed nr: 1
Resolution: 1.0, Modularity: 0.32026676770651435, Nr of communities: 9, Seed nr: 1
Resolution: 1.1, Modularity: 0.3156892906959601, Nr of communities: 12, Seed nr: 1
Resolution: 1.2, Modularity: 0.30841186600118586, Nr of communities: 16, Seed nr: 1
Resolution: 1.3, Modularity: 0.30517081587609013, Nr of communities: 16, Seed nr: 1
Resolution: 1.4, Modularity: 0.2989305631778372, Nr of communities: 21, Seed nr: 1
Resolution: 1.5, Modularity: 0.2911131798980443, Nr of communities: 25, Seed nr: 1
Resolution: 0.5, Modularity: 6.574162718404933e-06, Nr of communities: 2, Seed nr:

In [20]:
leiden_results = pd.DataFrame(zip(resolutions,community_nrs,modularity_scores, seed_nrs), columns = ['resolution', 'nr_communities', 'modularity', 'seed'])
leiden_results_grouped = leiden_results.groupby(['resolution']).mean().rename(columns = {'nr_communities':'nr_communities_av', 'modularity': 'modularity_av'}).drop(columns = ['seed'])
leiden_results_grouped['modularity_std'] = leiden_results.groupby(['resolution']).std()['modularity']
leiden_results_grouped['nr_communities_median'] = leiden_results.groupby(['resolution']).median()['nr_communities']
leiden_results_grouped['nr_communities_std'] = leiden_results.groupby(['resolution']).std()['nr_communities']
leiden_results_grouped

,nr_communities_av,modularity_av,modularity_std,nr_communities_median,nr_communities_std
resolution,,,,,
0.5,4.17,0.000054,0.000089,3.0,3.513481
0.6,4.91,0.248227,0.040622,5.0,2.030550
0.7,6.97,0.282977,0.025089,6.0,2.750409
0.8,7.98,0.302123,0.013324,8.0,1.543838
0.9,9.74,0.313811,0.005176,10.0,1.834683
1.0,10.62,0.318079,0.003512,10.0,1.354230
1.1,12.03,0.314888,0.003055,12.0,1.158674
1.2,14.77,0.308516,0.002811,15.0,1.376975
1.3,17.94,0.303757,0.002096,18.0,1.309021


In [24]:
leiden_results[leiden_results['resolution'] == 0.6].sort_values(['modularity'])

,resolution,nr_communities,modularity,seed
892,0.6,13,0.081787,82
166,0.6,5,0.162441,16
133,0.6,7,0.175172,13
1002,0.6,7,0.176352,92
617,0.6,7,0.176828,57
...,...,...,...,...
1035,0.6,3,0.299779,95
705,0.6,3,0.299801,65
89,0.6,3,0.300800,9
859,0.6,3,0.300909,79


In [22]:
leiden_results[leiden_results['resolution'] == 1].sort_values(['modularity'])

,resolution,nr_communities,modularity,seed
555,1.0,10,0.304916,51
1006,1.0,12,0.306094,92
566,1.0,14,0.309319,52
368,1.0,11,0.310021,34
357,1.0,9,0.311798,33
...,...,...,...,...
49,1.0,11,0.322035,5
962,1.0,9,0.322640,88
247,1.0,11,0.322817,23
16,1.0,10,0.323260,2


In [26]:
#export best modularity version of resolution 0.6
resolution = 0.6
seed = 71
leiden_partition_map = leiden(G_mc, resolution=resolution, random_seed = seed) # picking one of the seeds from the table above
community_nr = len(set(leiden_partition_map.values()))
modularity_score = modularity(G_mc, leiden_partition_map)
print(f"Resolution: {resolution}, Modularity: {modularity_score}, Nr of communities: {community_nr}, Seed nr: {seed}")


#Add the community information to nodes
for node, community in leiden_partition_map.items():
    G_mc.nodes[node]['community_small_res'] = community

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc.degree(weight='weight')).items():
    G_mc.nodes[node]['weighted_degree'] = weighted_degree

community_0 = []
community_1 = []
community_2 = []
    
for k,v in nx.get_node_attributes(G_mc, "community_small_res").items():
    if v == 0: 
        community_0.append(k)
    if v == 1: 
        community_1.append(k)
    if v == 2: 
        community_2.append(k)


# calculate within community weighted degrees
G_mc_com0 = G_mc.subgraph(community_0)
G_mc_com1 = G_mc.subgraph(community_1)
G_mc_com2 = G_mc.subgraph(community_2)

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com0.degree(weight='weight')).items():
    G_mc_com0.nodes[node]['weighted_degree_within_small_com'] = weighted_degree

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com1.degree(weight='weight')).items():
    G_mc_com1.nodes[node]['weighted_degree_within_small_com'] = weighted_degree

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com2.degree(weight='weight')).items():
    G_mc_com2.nodes[node]['weighted_degree_within_small_com'] = weighted_degree

Resolution: 0.6, Modularity: 0.30471943302283966, Nr of communities: 3, Seed nr: 71


In [28]:
#export best modularity version of resolution 1
resolution = 1
seed = 85

leiden_partition_map = leiden(G_mc, resolution=resolution, random_seed = seed) # picking one of the seeds from the table above
community_nr = len(set(leiden_partition_map.values()))
modularity_score = modularity(G_mc, leiden_partition_map)
print(f"Resolution: {resolution}, Modularity: {modularity_score}, Nr of communities: {community_nr}, Seed nr: {seed}")


#Add the community information to nodes
for node, community in leiden_partition_map.items():
    G_mc.nodes[node]['community_medium_res'] = community

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc.degree(weight='weight')).items():
    G_mc.nodes[node]['weighted_degree'] = weighted_degree

community_0 = []
community_1 = []
community_2 = []
community_3 = []
community_4 = []
community_5 = []
community_6 = []
community_7 = []
community_8 = []
community_9 = []

    
for k,v in nx.get_node_attributes(G_mc, "community_medium_res").items():
    if v == 0: 
        community_0.append(k)
    if v == 1: 
        community_1.append(k)
    if v == 2: 
        community_2.append(k)
    if v == 3: 
        community_3.append(k)
    if v == 4: 
        community_4.append(k)
    if v == 5: 
        community_5.append(k)
    if v == 6: 
        community_6.append(k)
    if v == 7: 
        community_7.append(k)
    if v == 8: 
        community_8.append(k)
    if v == 9: 
        community_9.append(k)


# calculate within community weighted degrees
G_mc_com0 = G_mc.subgraph(community_0)
G_mc_com1 = G_mc.subgraph(community_1)
G_mc_com2 = G_mc.subgraph(community_2)
G_mc_com3 = G_mc.subgraph(community_3)
G_mc_com4 = G_mc.subgraph(community_4)
G_mc_com5 = G_mc.subgraph(community_5)
G_mc_com6 = G_mc.subgraph(community_6)
G_mc_com7 = G_mc.subgraph(community_7)
G_mc_com8 = G_mc.subgraph(community_8)
G_mc_com9 = G_mc.subgraph(community_9)

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com0.degree(weight='weight')).items():
    G_mc_com0.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com1.degree(weight='weight')).items():
    G_mc_com1.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

#calculate weighted degree and add to G_mc
for node, weighted_degree in dict(G_mc_com2.degree(weight='weight')).items():
    G_mc_com2.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com3.degree(weight='weight')).items():
    G_mc_com3.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com4.degree(weight='weight')).items():
    G_mc_com4.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com5.degree(weight='weight')).items():
    G_mc_com5.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com6.degree(weight='weight')).items():
    G_mc_com6.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com7.degree(weight='weight')).items():
    G_mc_com7.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com8.degree(weight='weight')).items():
    G_mc_com8.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

for node, weighted_degree in dict(G_mc_com9.degree(weight='weight')).items():
    G_mc_com9.nodes[node]['weighted_degree_within_medium_com'] = weighted_degree

Resolution: 1, Modularity: 0.32343520568042095, Nr of communities: 10, Seed nr: 85


In [30]:
G_mc.nodes['hiv1']

{'label': 'hiv1',
 'community_small_res': 0,
 'weighted_degree': 4484.0,
 'weighted_degree_within_small_com': 3754.0,
 'community_medium_res': 6,
 'weighted_degree_within_medium_com': 3664.0}

In [32]:
#safe data
nx.write_gexf(G_mc, path+R"processed_data\co_word_network_+2tfidf_backbone_90_leiden_innerweighdeg_small_best.gexf")

In [33]:
pd.DataFrame.from_dict(dict(G_mc.nodes(data = True)), orient='index').drop(columns = ['label']).to_csv(path+'processed_data/co_word_network_+2tfidf_backbone_90_leiden_innerweighdeg_small_best.csv')

In [34]:
leiden_results.to_csv(path+R'processed_data\leiden_results.csv', index=False)
leiden_results_grouped.to_csv(path+R'processed_data\leiden_results_grouped.csv', index=False)